In [30]:
import numpy as np
import pandas as pd
import numpy as np
import os
import time
from google.cloud import bigquery
import pandas_gbq
import glob

In [16]:
client = bigquery.Client()

In [37]:
query = """
    SELECT ano, sigla_uf, ano_escolar, localizacao, dependencia_adm, docente_grupos, per_valor FROM `repositoriodedadosgpsp.Datalake.INEP_esforco_docente` where ano = 2022  order by sigla_uf
"""

In [38]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,ano,sigla_uf,ano_escolar,localizacao,dependencia_adm,docente_grupos,per_valor
0,2022,AC,Ensino Fundamental - Anos Finais,Rural,Municipal,Grupo 1,50.0
1,2022,AC,Ensino Fundamental - Todos os anos,Urbana,Estadual,Grupo 5,22.2
2,2022,AC,Ensino Fundamental - Todos os anos,Rural,Municipal,Grupo 2,0.0
3,2022,AC,Ensino Fundamental - Anos Finais,Rural,Estadual,Grupo 6,0.0
4,2022,AC,Ensino Fundamental - Todos os anos,Rural,Municipal,Grupo 2,0.0
...,...,...,...,...,...,...,...
3141763,2022,TO,Ensino Médio,Urbana,Estadual,Grupo 1,0.0
3141764,2022,TO,Ensino Fundamental - Anos Iniciais,Urbana,Privada,Grupo 2,0.0
3141765,2022,TO,Ensino Fundamental - Anos Finais,Urbana,Municipal,Grupo 3,NaN
3141766,2022,TO,Ensino Fundamental - Anos Finais,Urbana,Estadual,Grupo 3,100.0


In [39]:
df['docente_grupos'] = df['docente_grupos'].replace(to_replace=r'Grupo', value='Nível', regex=True)
df

,ano,sigla_uf,ano_escolar,localizacao,dependencia_adm,docente_grupos,per_valor
0,2022,AC,Ensino Fundamental - Anos Finais,Rural,Municipal,Nível 1,50.0
1,2022,AC,Ensino Fundamental - Todos os anos,Urbana,Estadual,Nível 5,22.2
2,2022,AC,Ensino Fundamental - Todos os anos,Rural,Municipal,Nível 2,0.0
3,2022,AC,Ensino Fundamental - Anos Finais,Rural,Estadual,Nível 6,0.0
4,2022,AC,Ensino Fundamental - Todos os anos,Rural,Municipal,Nível 2,0.0
...,...,...,...,...,...,...,...
3141763,2022,TO,Ensino Médio,Urbana,Estadual,Nível 1,0.0
3141764,2022,TO,Ensino Fundamental - Anos Iniciais,Urbana,Privada,Nível 2,0.0
3141765,2022,TO,Ensino Fundamental - Anos Finais,Urbana,Municipal,Nível 3,NaN
3141766,2022,TO,Ensino Fundamental - Anos Finais,Urbana,Estadual,Nível 3,100.0


In [40]:
df = df.rename(columns={'docente_grupos': 'docente_niveis'})
df

,ano,sigla_uf,ano_escolar,localizacao,dependencia_adm,docente_niveis,per_valor
0,2022,AC,Ensino Fundamental - Anos Finais,Rural,Municipal,Nível 1,50.0
1,2022,AC,Ensino Fundamental - Todos os anos,Urbana,Estadual,Nível 5,22.2
2,2022,AC,Ensino Fundamental - Todos os anos,Rural,Municipal,Nível 2,0.0
3,2022,AC,Ensino Fundamental - Anos Finais,Rural,Estadual,Nível 6,0.0
4,2022,AC,Ensino Fundamental - Todos os anos,Rural,Municipal,Nível 2,0.0
...,...,...,...,...,...,...,...
3141763,2022,TO,Ensino Médio,Urbana,Estadual,Nível 1,0.0
3141764,2022,TO,Ensino Fundamental - Anos Iniciais,Urbana,Privada,Nível 2,0.0
3141765,2022,TO,Ensino Fundamental - Anos Finais,Urbana,Municipal,Nível 3,NaN
3141766,2022,TO,Ensino Fundamental - Anos Finais,Urbana,Estadual,Nível 3,100.0


Subindo para o GBQ

In [41]:
dataset_ref = client.dataset('percepcao_mensuracao_qualidade_estado')

In [42]:
df.dtypes

ano                  Int64
sigla_uf            object
ano_escolar         object
localizacao         object
dependencia_adm     object
docente_niveis      object
per_valor          float64
dtype: object

In [45]:
schema = [bigquery.SchemaField('ano', 'INTEGER', description= 'Ano de referência da observação'),
          bigquery.SchemaField('sigla_uf', 'STRING', description= 'Sigla da Unidade da Federação referente a observação'),
          bigquery.SchemaField('localizacao', 'STRING', description= 'Se a área da observação é urbana ou rural'),
          bigquery.SchemaField('dependencia_adm', 'STRING', description= 'Dependência admnistrativa da observação (privada ou pública)'),
          bigquery.SchemaField('ano_escolar', 'STRING', description= 'Ano escolar referente ao qual docente leciona'),
          bigquery.SchemaField('docente_niveis', 'STRING', description= 'Cinco categorias de adequação da formação dos docentes em relação à disciplina que leciona'),
          bigquery.SchemaField('per_valor', 'FLOAT', description= 'Percentual de docentes que lecionam na respectiva agregação em cada um dos níveis da escala do indicador'),
]

In [46]:
table_ref = dataset_ref.table('INEP_esforco_docente_atual_temporal')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=e7af07ac-619c-4f11-a1f7-fb76374a152c>